In [41]:
import math
import numpy as np

In [42]:
from qiskit.quantum_info import Pauli, Operator
from qiskit.quantum_info import SparsePauliOp

In [56]:
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)

num_nodes = 2
boundary_condition = BoundaryCondition.OPEN
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

In [57]:
#Bose-Hubbard Hamiltonian of a system of bosons 
#binary mapping

class BoseHubbardHamiltonian:
        
    def __init__(self, n_sites, single_site_qubits, geometry):
        
        # n_sites (int) = number of sites in the lattice
        # single_site_qubits (int) = number of qubits used to describe each site
        # geometry (LineLattice or another type of lattice) = how the sites are distributed in space and their connectivity 
              
        self.n_sites = n_sites
        self.ss_q = single_site_qubits
        self.geometry = geometry.to_adjacency_matrix() # the adjacency matrix gives informations about the connectivity of the sites


    #creation, annihilation, number and identity operators on single_site_qubits number of qubits
    def second_quantization_ops(self):
       
        dim =  2**(self.ss_q)
        create_matrix = np.zeros((dim,dim),dtype=float)
        
        for i in range(dim-1): 
            create_matrix[i+1][i] = math.sqrt(i+1)
        
        I_q_matrix = np.eye(dim, dtype=float)
        I_q_Op = Operator(I_q_matrix)
        
        # from matrix to Operator and from Operator to SparsePauliOp
        createOp = Operator(create_matrix)
        self.create = SparsePauliOp.from_operator(createOp)
     
        self.annihilate = self.create.adjoint()
        self.number = self.create.compose(self.annihilate).simplify()
        
        self.I_q =  SparsePauliOp.from_operator(I_q_Op)

        
    #removal of the diagonal terms in the adjacency matrix of the lattice
    def lattice_connectivity(self):

        for i in range(self.n_sites):
            for j in range(self.n_sites):
                if i == j:
                    self.geometry[i][j] = 0
        

    #kinetic energy operator
    def kinetic_energy_op(self): 

        Op = []
        for i in range(self.n_sites): 
            Op.append(self.I_q)

        kin = []
        for i in range(self.n_sites):
            for j in range(self.n_sites):
       
                if geometry[i][j] == 1:
                    Op[i] = self.create
                    Op[j] = self.annihilate
                      
                    Operator = Op[0]
                    k = 1
                    while k < self.n_sites:
                        Operator = Operator^Op[k]
                        k = k+1

                    kin.append(Operator)

                    Op[i] = self.I_q
                    Op[j] = self.I_q
            

        self.Kinetic = sum(kin).simplify()
        return self.Kinetic
            
    #potential energy (total interaction term) operator    
    def potential_energy_op(self):
        
        I_n_list = []
        Op = []
        for i in range(self.n_sites):
            I_n_list.append(self.I_q)
            Op.append(self.I_q)

        #identità per n siti con ss_q qubit per sito
        I_n = I_n_list[0]
        i = 1
        while i < self.n_sites:
            I_n = I_n^I_n_list[i]
            i = i+1
        
        self.I_n = I_n
        
        number_tot = []
        potent = []
        for i in range(self.n_sites): 
            Op[i] = self.number
       
            Operator = Op[0]
            k = 1
            while k < self.n_sites:
                Operator = Operator^Op[k]
                k = k+1

            number_tot.append(Operator)
                    
            Op[i] = self.I_q
                    
        for i in range(self.n_sites):
            potent.append( number_tot[i].compose(number_tot[i] - self.I_n))

        self.Potential = sum(potent).simplify()

        return self.Potential

    def get_H(self, J, U):
        
        BoseHubbardHamiltonian.second_quantization_ops(self)
        BoseHubbardHamiltonian.lattice_connectivity(self)
        BoseHubbardHamiltonian.kinetic_energy_op(self)
        BoseHubbardHamiltonian.potential_energy_op(self)
             
        self.U = U
        self.J = J
        H =  J*self.Kinetic + U*self.Potential
        return H
        

In [58]:
H_instance = BoseHubbardHamiltonian(2, 2, line_lattice)
H = H_instance.get_H(1,1)
print(H)

SparsePauliOp(['IXIX', 'IXXX', 'IXYY', 'IXZX', 'IYIY', 'IYXY', 'IYYX', 'IYZY', 'XXIX', 'XXXX', 'XXYY', 'XXZX', 'XYIY', 'XYXY', 'XYYX', 'XYZY', 'YXIY', 'YXXY', 'YXYX', 'YXZY', 'YYIX', 'YYXX', 'YYYY', 'YYZX', 'ZXIX', 'ZXXX', 'ZXYY', 'ZXZX', 'ZYIY', 'ZYXY', 'ZYYX', 'ZYZY', 'IIII', 'IZII', 'ZZII', 'ZIII', 'IIIZ', 'IIZZ', 'IIZI'],
              coeffs=[ 0.9330127 +0.j,  0.48296291+0.j,  0.48296291+0.j, -0.25      +0.j,
  0.9330127 +0.j, -0.48296291+0.j,  0.48296291+0.j, -0.25      +0.j,
  0.48296291+0.j,  0.25      +0.j,  0.25      +0.j, -0.12940952+0.j,
 -0.48296291+0.j,  0.25      +0.j, -0.25      +0.j,  0.12940952+0.j,
  0.48296291+0.j, -0.25      +0.j,  0.25      +0.j, -0.12940952+0.j,
  0.48296291+0.j,  0.25      +0.j,  0.25      +0.j, -0.12940952+0.j,
 -0.25      +0.j, -0.12940952+0.j, -0.12940952+0.j,  0.0669873 +0.j,
 -0.25      +0.j,  0.12940952+0.j, -0.12940952+0.j,  0.0669873 +0.j,
  4.        +0.j,  1.        +0.j, -2.        +0.j, -1.        +0.j,
  1.        +0.j, -2.        +

In [ ]:
#CONTROLLI

In [4]:
#creation of the square matrix that represents the bosonic creation operator

n_qubits = 2 #single site qubits

dim =  2**(n_qubits)
create_matrix = np.zeros((dim,dim),dtype=float)

for i in range(dim-1): 
    create_matrix[i+1][i] = math.sqrt(i+1)


print(create_matrix) 

[[0.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.         1.41421356 0.         0.        ]
 [0.         0.         1.73205081 0.        ]]


In [5]:
#identità su single_sites_qubits

I_q_matrix = np.eye(dim, dtype=float)
print(I_q_matrix)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [29]:
#from matrix to operator

createOp = Operator(create_matrix)
create = SparsePauliOp.from_operator(createOp)

annihilate = create.adjoint()

number = create.compose(annihilate).simplify()

I_q_Op = Operator(I_q_matrix)
I_q =  SparsePauliOp.from_operator(I_q_Op)

print(create)
print("\n",annihilate)
print("\n", number)
print("\n",I_q)

SparsePauliOp(['IX', 'IY', 'XX', 'XY', 'YX', 'YY', 'ZX', 'ZY'],
              coeffs=[ 0.6830127 +0.j        ,  0.        -0.6830127j ,  0.35355339+0.j        ,
  0.        +0.35355339j,  0.        -0.35355339j,  0.35355339+0.j        ,
 -0.1830127 +0.j        ,  0.        +0.1830127j ])

 SparsePauliOp(['IX', 'IY', 'XX', 'XY', 'YX', 'YY', 'ZX', 'ZY'],
              coeffs=[ 0.6830127 -0.j        ,  0.        +0.6830127j ,  0.35355339-0.j        ,
  0.        -0.35355339j,  0.        +0.35355339j,  0.35355339-0.j        ,
 -0.1830127 -0.j        ,  0.        -0.1830127j ])

 SparsePauliOp(['II', 'IZ', 'ZZ'],
              coeffs=[ 1.5+0.j,  0.5+0.j, -1. +0.j])

 SparsePauliOp(['II'],
              coeffs=[1.+0.j])


In [59]:
geometry = line_lattice.to_adjacency_matrix()
print(geometry)
struct = geometry.shape
n_sites = struct[0] 
print("\n", struct)

for i in range(n_sites):
    for j in range(n_sites):
        if i == j:
            geometry[i][j] = 0

print("\n", geometry) 

[[1. 1.]
 [1. 1.]]

 (2, 2)

 [[0. 1.]
 [1. 0.]]


In [60]:
#test
n = 3
Operator = Op[0]
i = 1
while i < n:
    Operator = Operator^Op[i]
    i = i+1

In [36]:
print(Operator)

SparsePauliOp(['IIIIII'],
              coeffs=[1.+0.j])


In [62]:
#test
#kinetic energy - any n_sites

n_sites = struct[0] 
Op = []
for i in range(n_sites): 
    Op.append(I_q)

kin = []
for i in range(n_sites):
    for j in range(n_sites):
       
        if geometry[i][j] == 1:
            Op[i] = create
            Op[j] = annihilate
                      
            Operator = Op[0]
            k = 1
            while k < n_sites:
                Operator = Operator^Op[k]
                k = k+1

            kin.append(Operator)

            Op[i] = I_q
            Op[j] = I_q
            

Kinetic = sum(kin).simplify()
print(Kinetic)


SparsePauliOp(['IXIX', 'IXXX', 'IXYY', 'IXZX', 'IYIY', 'IYXY', 'IYYX', 'IYZY', 'XXIX', 'XXXX', 'XXYY', 'XXZX', 'XYIY', 'XYXY', 'XYYX', 'XYZY', 'YXIY', 'YXXY', 'YXYX', 'YXZY', 'YYIX', 'YYXX', 'YYYY', 'YYZX', 'ZXIX', 'ZXXX', 'ZXYY', 'ZXZX', 'ZYIY', 'ZYXY', 'ZYYX', 'ZYZY'],
              coeffs=[ 0.9330127 +0.j,  0.48296291+0.j,  0.48296291+0.j, -0.25      +0.j,
  0.9330127 +0.j, -0.48296291+0.j,  0.48296291+0.j, -0.25      +0.j,
  0.48296291+0.j,  0.25      +0.j,  0.25      +0.j, -0.12940952+0.j,
 -0.48296291+0.j,  0.25      +0.j, -0.25      +0.j,  0.12940952+0.j,
  0.48296291+0.j, -0.25      +0.j,  0.25      +0.j, -0.12940952+0.j,
  0.48296291+0.j,  0.25      +0.j,  0.25      +0.j, -0.12940952+0.j,
 -0.25      +0.j, -0.12940952+0.j, -0.12940952+0.j,  0.0669873 +0.j,
 -0.25      +0.j,  0.12940952+0.j, -0.12940952+0.j,  0.0669873 +0.j])


In [63]:
#test
#potential energy - any n_sites

n_sites = struct[0] 

I_n_list = []
Op = []
for i in range(n_sites):
    I_n_list.append(I_q)
    Op.append(I_q)

I_n = I_n_list[0]
i = 1
while i < n_sites:
    I_n = I_n^I_n_list[i]
    i = i+1

number_tot = []
potent = []
           
for i in range(n_sites): 
        Op[i] = number
       
        Operator = Op[0]
        k = 1
        while k < n_sites:
            Operator = Operator^Op[k]
            k = k+1

        number_tot.append(Operator)
                    
        Op[i] = I_q
                    
for i in range(n_sites):
        potent.append( number_tot[i].compose(number_tot[i] - I_n))

Potential_op = sum(potent).simplify()
print(Potential_op)

SparsePauliOp(['IIII', 'IZII', 'ZZII', 'ZIII', 'IIIZ', 'IIZZ', 'IIZI'],
              coeffs=[ 4.+0.j,  1.+0.j, -2.+0.j, -1.+0.j,  1.+0.j, -2.+0.j, -1.+0.j])
